# Formatting Outputs 



When you return a value or a display a value in a .NET notebook, the default formatting uses HTML to render the object. For many values, a table structure is used by default. For example:

In [ ]:
display ["hello"; "world"]

[ 1 .. 4 ]

## Switching to plain text only

If you prefer, you can switch to plain text by default.  The simplest way to do this is to register `"text/plain"` as the default mime type for all objects and switch the `%A` printing.  

In [ ]:
Formatter.SetPreferredMimeTypeFor(typeof<obj>, "text/plain")
Formatter.Register(fun (x:obj) (writer: TextWriter) -> fprintfn writer "%120A" x )

[ 1 .. 4 ]

Now reset back to defaults:

In [ ]:
Formatter.ResetToDefault()

## HTML formatting

### Strings

By default multi-line content does not have formatting preserved.  For example:

In [ ]:
[ sprintf "%120A" [ for i in 1 .. 30 -> [ 1 .. i ]] ]

You can use styling to preserve the preformatting of such content.

In [ ]:
CSS ".dni-plaintext { text-align: left; white-space: pre; font-family: monospace; });"

Note that `Formatter.ResetToDefault()` doesn't undo any CSS changes. You can undo this change with:

In [ ]:
CSS ".dni-plaintext { text-align: inherit; white-space: inherit; font-family: inherit; });"

### Objects

The default formatting behavior for many objects is to produce a table showing their properties and the values of those properties.

In [ ]:
type Person = { FirstName: string; LastName: string; Age: int }

// Evaluate a new person
{ FirstName = "Mitch"; LastName = "Buchannon"; Age = 42 }

###  Sequences

When you have a collection of objects, you can see the values listed for each item in the collection:

In [ ]:
[
    { FirstName = "Mitch"; LastName = "Buchannon"; Age = 42 }
    { FirstName = "Hobie "; LastName = "Buchannon"; Age = 23 }
    { FirstName = "Summer"; LastName = "Quinn"; Age = 25 }
    { FirstName = "C.J."; LastName = "Parker"; Age = 23 }
]


### Dictionaries

Similarly to the behavior for `IEnumerable` objects, you'll also see table output for dictionaries, but for each value in the dictionary, the key is provided rather than the index within the collection.

In [ ]:
// Cannot simply use 'dict' here, see https://github.com/dotnet/interactive/issues/12

dict [("zero", 0); ("one", 1); ("two", 2)]


###  Nested objects

Now let's try something a bit more complex. Let's look at a graph of objects. 

We'll redefine the `Person` class to allow a reference to a collection of other `Person` instances.

In [ ]:
type Person =
    { FirstName: string
      LastName: string
      Age: int
      Friends: ResizeArray<Person> }

let mitch = { FirstName = "Mitch"; LastName = "Buchannon"; Age = 42; Friends = ResizeArray() }
let hobie = { FirstName = "Hobie "; LastName = "Buchannon"; Age = 23; Friends = ResizeArray() }
let summer =  { FirstName = "Summer"; LastName = "Quinn"; Age = 25; Friends = ResizeArray() }

mitch.Friends.AddRange([ hobie; summer ])
hobie.Friends.AddRange([ mitch; summer ])
summer.Friends.AddRange([ mitch; hobie ])

let people = [ mitch; hobie; summer ]
display people

That's a bit hard to read, right?  The defaut formatting behaviors are not always as useful as they might be. In order to give you more control in these kinds of cases, formatters can be customized from within the .NET notebook.

## Customizing

### Registering plain text formatters

Let's clean up the output above by customizing the formatter for the `Person.Friends` property, which is creating a lot of noise. 

The way to do this is to use the `Formatter` API. This API lets you customize the formatting for a specific type. Since `Person.Friends` is a sequence of `Person` objects, i.e. type `seq<Person>`, we can register a custom formatter for that type to change the output. Let's just list their first names:

In [ ]:
Formatter.Register<seq<Person>>(
    mimeType = "text/plain",
    formatter = Func<_,_,_,_>(fun context people (writer: TextWriter) ->
        for person in people do
            writer.Write(person.FirstName)
            writer.Write(" ")
        true))


Now display the `people` data again:

In [ ]:
people

You might have noticed that `people` is of type `ResizeArray<Person>`, but the table output still includes columns for `LastName`, `Age`, and `Friends`. What's going on here? Notice that the custom formatter we just registered was registered for the mime type `"text/plain"`. The top-level formatter that's used when we call `display` requests output of mime type `"text/html"` and the nested objects are formatted using `"text/plain"`. It's the nested objects, not the top-level HTML table, that's using the custom formatter here. If you'd like to change the default mime type, use `SetPreferredMimeTypeFor`.



### Registering HTML formatters

To replace the default HTML view for a particular type, you can register a formatter for the `"text/html"` mime type. 

The HTML is specified using the HTML DSL similar to Giraffe's ViewEngine.  Note: In this example we put the formatter registration in a module to prevent opening the `Html` DSL everywhere.

In [ ]:
module PersonHtmlFormatter = 
    
    // Locally open the F# HTML DSL.
    open Html

    Formatter.Register<seq<Person>>(
        mimeType = "text/html",
        formatter = Func<_,_,_,_>(fun (context: FormatContext) (people: seq<Person>) (writer: TextWriter) ->
            table [] [
              thead [ _style ["color: blue"]] [
                th [] [ str "First Name" ]
                th [] [ str "Last Name" ]
                th [] [ str "Age" ]
              ]
              tbody [_style ["color: darkolivegreen"]] [
                for p in people do
                  tr [] [
                    td [] [ str p.FirstName ]
                    td [] [ str p.LastName ]
                    td [] [ str (string p.Age) ; str " years" ] 
                  ]
              ]
            ]
            |> writer.Write
            true))


Now display the `people` data again:

In [ ]:
people

### Registering formatters for generic types




Formatters can be specified by using a generic type definition as a key, for example:



In [ ]:
Formatter.Register(``type`` = typedefof<System.Collections.Generic.List<_>>,
    formatter = Action<obj,TextWriter>(fun (value: obj) (writer: TextWriter) ->
        writer.Write("quack")
    ), 
    mimeType = "text/html")

Then

In [ ]:
new System.Collections.Generic.List<int>()

Reflection can then be used to operate on the object at its more specific type.